In [30]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [31]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [32]:
from torchvision import datasets, transforms
data_path = '../../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [33]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [34]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']

In [35]:
cifar2 = [(img, label_map[label])
          for img, label in cifar10 
          if label in [0, 2]]

cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

In [36]:
print(len(cifar10), len(cifar10_val))
print(len(cifar2), len(cifar2_val))

50000 10000
10000 2000


In [37]:
device = (torch.device('cuda') if torch.cuda.is_available()
            else torch.device('cpu'))

print(device)

cuda


In [38]:
cifar2 = [(img.to(device),
           torch.tensor(label).to(device)) for img, label in cifar2]
cifar2_val = [(img.to(device), 
               torch.tensor(label).to(device)) for img, label in cifar2_val]

In [39]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

## 原

---

In [28]:
model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 2)).to(device)

learning_rate = 1e-1

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

n_epochs = 100

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.710599
Epoch: 1, Loss: 0.398297
Epoch: 2, Loss: 0.265748
Epoch: 3, Loss: 0.477997
Epoch: 4, Loss: 0.347228
Epoch: 5, Loss: 0.298988
Epoch: 6, Loss: 0.326244
Epoch: 7, Loss: 0.267870
Epoch: 8, Loss: 0.296569
Epoch: 9, Loss: 0.216771
Epoch: 10, Loss: 0.340905
Epoch: 11, Loss: 0.412462
Epoch: 12, Loss: 0.094640
Epoch: 13, Loss: 0.106730
Epoch: 14, Loss: 0.106087
Epoch: 15, Loss: 0.107549
Epoch: 16, Loss: 0.259040
Epoch: 17, Loss: 0.128283
Epoch: 18, Loss: 0.120966
Epoch: 19, Loss: 0.312628
Epoch: 20, Loss: 0.254893
Epoch: 21, Loss: 0.214502
Epoch: 22, Loss: 0.015418
Epoch: 23, Loss: 0.130515
Epoch: 24, Loss: 0.221259
Epoch: 25, Loss: 0.003098
Epoch: 26, Loss: 0.035281
Epoch: 27, Loss: 0.054112
Epoch: 28, Loss: 0.001782
Epoch: 29, Loss: 0.002782
Epoch: 30, Loss: 0.008732
Epoch: 31, Loss: 0.201580
Epoch: 32, Loss: 0.025544
Epoch: 33, Loss: 0.048837
Epoch: 34, Loss: 0.041420
Epoch: 35, Loss: 0.002884
Epoch: 36, Loss: 0.011901
Epoch: 37, Loss: 0.028004
Epoch: 38, Loss: 0.001

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

In [ ]:
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

## MSEloss

---

In [44]:
outputs = model(imgs.view(imgs.shape[0], -1))

In [46]:
outputs.shape, labels.shape

(torch.Size([64, 2]), torch.Size([64]))

In [48]:
labels

tensor([1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0], device='cuda:0')

In [47]:
outputs

tensor([[0.5359, 0.4641],
        [0.4836, 0.5164],
        [0.5043, 0.4957],
        [0.4864, 0.5136],
        [0.4710, 0.5290],
        [0.4459, 0.5541],
        [0.5290, 0.4710],
        [0.4828, 0.5172],
        [0.4198, 0.5802],
        [0.4818, 0.5182],
        [0.4852, 0.5148],
        [0.5307, 0.4693],
        [0.4654, 0.5346],
        [0.4806, 0.5194],
        [0.4740, 0.5260],
        [0.5418, 0.4582],
        [0.4456, 0.5544],
        [0.4469, 0.5531],
        [0.4351, 0.5649],
        [0.5383, 0.4617],
        [0.5484, 0.4516],
        [0.4696, 0.5304],
        [0.5315, 0.4685],
        [0.4968, 0.5032],
        [0.4634, 0.5366],
        [0.4909, 0.5091],
        [0.4730, 0.5270],
        [0.4619, 0.5381],
        [0.4765, 0.5235],
        [0.5477, 0.4523],
        [0.4784, 0.5216],
        [0.5023, 0.4977],
        [0.4612, 0.5388],
        [0.5264, 0.4736],
        [0.5429, 0.4571],
        [0.4825, 0.5175],
        [0.4882, 0.5118],
        [0.5456, 0.4544],
        [0.4

In [ ]:
# 需要先换label成outputs对应的1,0

In [43]:
model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 2),
            nn.Softmax(dim=1)).to(device)

learning_rate = 1e-1

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.MSELoss()

n_epochs = 100

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

RuntimeError: The size of tensor a (2) must match the size of tensor b (64) at non-singleton dimension 1